In [1]:
import pysam
import os
import pandas as pd
import numpy as np

In [2]:
def denovo_links(bam_file, out_name):
    """
    Function to be applied to sorted bam file
    """
    print('Starting ', bam_file)
    output_file = open(out_name,'w')
    
    samfile = pysam.AlignmentFile(bam_file, 'rb')
    count = 0
    genomes = {'pB10::rfp_putative':'pB10', 'NC_000913.3_gfp':'E.Coli'}
    pB10_pB10 = 0
    pB10_ecoli = 0
    pB10_unaligned = 0
    unaligned_unaligned = 0
    
    pairs = []
    
    #lists that store the location on the pB10 genome of each aligned read
    pB10_pB10_loc = []
    pB10_unknown_loc = []
    pB10_ecoli_loc = []
    
    global p_unaligned
    global p_p
    global p_ecoli
    p_unaligned = [0]*pb10_len
    p_p = [0]*pb10_len
    p_ecoli = [0]*pb10_len
    
    info_df = pd.DataFrame(columns = ['seqid', 'pB10_start','read_length'])

    for read in samfile.fetch(until_eof=True):
        count += 1
        genome = ''
        name = read.query_name
        cigar = ''
        seq = ''
        loc = ''
        if read.is_unmapped:
            genome = 'unaligned'
            cigar = read.cigarstring
            seq = read.query_sequence
            length = len(seq)
        if not read.is_unmapped:
            genome = genomes.get(read.reference_name)
            #cigar = read.cigarstring
            seq = read.query_sequence
            match = str(len(seq))+'M'
            loc = read.reference_start
            length = len(seq)
            #if cigar != match:
            #    genome = 'unaligned'
        pairs.append([name, genome, loc, length, seq, read.get_tags()])
        if count % 2 == 0:
            r1_multimapped = check_alt(pairs[0][1], pairs[0][5], pairs[0][3])
            r2_multimapped = check_alt(pairs[1][1], pairs[1][5], pairs[1][3])
            if len(r1_multimapped) > 1 or len(r2_multimapped) >1:
                multimapped += 1
                pairs = []
            else:
                links = pairs[0][1] + pairs[1][1]
                if links == 'pB10pB10':
                    pB10_pB10 += 1
                    update_counts('plasmid','plasmid', [pairs[0][2], pairs[1][2]], [pairs[0][3], pairs[1][3]])
                    pB10_pB10_loc.append(pairs[0][2])
                    pB10_pB10_loc.append(pairs[1][2])
                if links == 'pB10unaligned':
                    pB10_unaligned += 1
                    update_counts('plasmid', 'unaligned', pairs[0][2], pairs[0][3])
                    pB10_unknown_loc.append(pairs[0][2])
                    info_df = info_df.append({'seqid':pairs[1][0], 'pB10_start':pairs[0][2], 'read_length':pairs[0][3]}, ignore_index=True)
                    output_file.write(">" + pairs[1][0] + '\n' + pairs[1][4] + '\n')
                if links == 'unalignedpB10':
                    pB10_unaligned += 1
                    update_counts('unaligned', 'plasmid', pairs[1][2], pairs[1][3])
                    pB10_unknown_loc.append(pairs[1][2])
                    info_df = info_df.append({'seqid':pairs[0][0], 'pB10_start':pairs[1][2], 'read_length':pairs[1][3]}, ignore_index=True)
                    output_file.write(">" + pairs[0][0] + '\n' + pairs[0][4] + '\n')
                if links == 'pB10E.Coli':
                    pB10_ecoli += 1
                    update_counts('plasmid', 'ecoli', pairs[0][2], pairs[0][3])
                    pB10_ecoli_loc.append(pairs[0][2])
                if links == 'E.ColipB10':
                    pB10_ecoli += 1
                    update_counts('ecoli', 'plasmid', pairs[1][2], pairs[1][3])
                    pB10_ecoli_loc.append(pairs[1][2])
                if links == 'E.ColiE.Coli':
                    ecoli_ecoli += 1
                if links == 'unalignedunaligned':
                    unaligned_unaligned += 1
                pairs = []
    output_file.close()
    print(bam_file, 'Completed')
    print(np.array([count/2, pB10_pB10, pB10_ecoli, pB10_unaligned, unaligned_unaligned, multimapped], '\n'))
    return(np.array([[count/2, pB10_pB10, pB10_ecoli, pB10_unaligned, unaligned_unaligned, multimapped],
                     [p_unaligned, p_p, p_ecoli],
                     [pB10_unknown_loc, pB10_pB10_loc, pB10_ecoli_loc]]), 
           info_df)

def update_counts(ref, link, start_index, length):
    """Function for updating the coverage list"""
    global p_putida
    global p_ecoli
    global p_p
    
    if ref == 'plasmid':
        if link == 'ecoli':
            s = start_index
            e = start_index + length
            for i in range(s, e):
                if i < pb10_len:
                    p_ecoli[i] += 1
                else:
                    i = i - pb10_len
                    p_ecoli[i] += 1 

    if link == 'plasmid':
        if ref == 'ecoli':
            s = start_index
            e = start_index + length
            for i in range(s, e):
                if i < pb10_len:
                    p_ecoli[i] += 1
                else:
                    i = i - pb10_len
                    p_ecoli[i] += 1 
                    
    if ref == 'plasmid' and link == 'plasmid':                  
        for i in range(0,2):
            s = start_index[i]
            e = start_index[i] + length[i]
            for i in range(s, e):
                if i < pb10_len:
                    p_p[i] += 1
                else:
                    i = i - pb10_len
                    p_p[i] += 1
                    
def check_alt(aligned, tags, length):
    """
    This function uses the alignment tags to determine if a read was
    mapped perfectly to multiple references. The output is a list of 
    references to which the read mapped.
    """
    genomes = [aligned]
    for tag in tags:
        if tag[0] == 'XA' or tag[0] == 'SA':
            tag = list(tag)
            others = tag[1].split(';')[:-1]
            fmatch = []
            for alt in others:
                info = alt.split(',')
                if info[-2] == str(length)+'M':
                    fmatch.append(info[0])
            for i in fmatch:
                genomes.append(i)
    return(list(set(genomes)))

In [ ]:
os.chdir('/mnt/ceph/cast9836/00_projects/hic_targetcapture/05_bwa_alignment/de_novo/')

pb10_len = 68345
p_p = [0]*pb10_len
p_ecoli = [0]*pb10_len
p_putida = [0]*pb10_len

Bplus_allden, Bplus_df = denovo_links('C+_aligned_sorted.bam', '../../06_output/C+_pothosts.fasta')
#Cplus_allden, Cplus_df = denovo_links('D+_aligned_sorted.bam', '../../06_output/D+_pothosts.fasta')
#Dplus_allden, Dplus_df = denovo_links('E+_aligned_sorted.bam', '../../06_output/E+_pothosts.fasta')
#Eplus_allden, Eplus_df = denovo_links('F+_aligned_sorted.bam', '../../06_output/F+_pothosts.fasta')
#Fplus_allden, Fplus_df = denovo_links('G+_aligned_sorted.bam', '../../06_output/G+_pothosts.fasta')
#ctrl1plus_allden, ctrlplus_df = denovo_links('ctrl1+_ecoli_aligned_sorted.bam', '../../06_output/de_novo/ctrl1+_pothosts.fasta')

Starting  C+_aligned_sorted.bam


In [33]:
Bplus_df

NameError: name 'Bplus_df' is not defined

In [ ]:
import os
import pandas as pd
os.chdir('/mnt/ceph/cast9836/00_projects/hic_targetcapture/06_output/de_novo/')

cplus_taxon = pd.read_csv('C+_lineage.txt', sep='\t')
cplus_kraken = pd.read_csv('C+_out.kraken', sep='\t', header=None)
cplus_taxon['seqid'] = cplus_kraken[1]